In [3]:
import sqlite3
import pandas as pd
import numpy as np 
import torchvision
import torch
from PIL import Image
import asyncio
import io
import os
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from torch.utils.data import Subset
from deepmorpho.dl_folder.data_classes.morpho_dataset import EmbryoDataset
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from torchvision import transforms
from torch.utils.data import DataLoader
import timm 

In [1]:
def connect_to_database(db_path):
    """Подключение к базе данных SQLite."""
    try:
        conn = sqlite3.connect(db_path)
        print("Успешно подключились к базе данных")
        return conn
    except sqlite3.Error as e:
        print(f"Ошибка при подключении к SQLite: {e}")
        return None

In [3]:
def extract_data(conn):
    try:
        cursor = conn.cursor()
        query = "SELECT wtf_wtl_id, wtf_frame, wtf_rcnn_text FROM well_timeline_frames;"
        cursor.execute(query)
        data = cursor.fetchall()
        images = []
        annotations = []
        for wtf_wtl_id, blob_data, json_data in data:
            image_bytes = io.BytesIO(blob_data)
            image = Image.open(image_bytes)
            try:
                annotation = json.loads(json_data)
                images.append((wtf_wtl_id, image))
                annotations.append((wtf_wtl_id, annotation))
            except TypeError:
                continue
        return images, annotations
    except sqlite3.Error as e:
        print(f"Ошибка при выполнении SQL: {e}")
    except IOError as e:
        print(f"Ошибка при открытии изображения: {e}")







In [2]:
db_path = 'C:\Work\CLASSES\SPRING2024\DeepMorphoDynamics\deepmorpho\so_deep.db'  
conn = connect_to_database(db_path)
if conn:
    images, annotations = extract_data(conn)
    conn.close()
else:
    print("Не удалось подключиться к базе данных.")

NameError: name 'sqlite3' is not defined

In [5]:
result = []

for annotation in annotations:
    id = annotation[0]  
    data = annotation[1]  
    highest_prediction = max(data['predictions'], key=lambda x: x['prediction'])
    label = highest_prediction['label']
    prediction = highest_prediction['prediction']
    bbox = data['bboxes'][0]  
    result.append({'ID': id, 'label': label, 'prediction': prediction, 'bbox': bbox})

print(result[0])
images[0]


{'ID': 461, 'label': 'BLFL', 'prediction': 0.6353023648262024, 'bbox': [210, 118, 425, 323]}


(461, <PIL.BmpImagePlugin.BmpImageFile image mode=L size=500x500>)

In [6]:
all_classes = set()

for annotation in annotations:
    data = annotation[1]  
    for prediction in data['predictions']:
        all_classes.add(prediction['label'])

len(all_classes)

15

Now, we prepare dataset.

In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
])

In [8]:
dataset = EmbryoDataset(result, images, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
train_indices, test_indices, _, _ = train_test_split(
    range(len(dataset)),
    dataset.labels, 
    test_size=0.2,
    random_state=42
)

Now, VIS transformer. 

In [9]:
train_subset = Subset(dataset, train_indices)
test_subset = Subset(dataset, test_indices)

train_dataloader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_subset, batch_size=32, shuffle=False)

model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=len(all_classes))
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()


In [10]:
def evaluate_accuracy(dataloader, model):
    correct = 0
    total = 0
    model.eval()  
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    model.train()  
    return 100 * correct / total

In [ ]:
num_epochs = 3

model_dir = 'saved_models'
os.makedirs(model_dir, exist_ok=True)
for epoch in range(num_epochs):
    print(f'Epoch n: {epoch+1}/{num_epochs}')
    print('-' * 10)

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        print(f'Working on batch  {batch_idx+1}/{len(train_dataloader)}')


        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Loss: {loss.item()}')
    torch.save(model.state_dict(), f'{model_dir}/model_epoch_{epoch+1}.pth')
    accuracy = evaluate_accuracy(test_dataloader, model)
    print(f'Epoch [{epoch+1}/3]'
          f' Accuracy: {accuracy}%') 


Epoch n: 1/3
----------
Working on batch  1/290
Loss: 2.834705114364624
Working on batch  2/290
Loss: 1.9630963802337646
Working on batch  3/290
Loss: 4.22343635559082
Working on batch  4/290
Loss: 2.6772537231445312
Working on batch  5/290
Loss: 1.269192099571228
Working on batch  6/290
Loss: 1.5225821733474731
Working on batch  7/290
Loss: 1.7879706621170044
Working on batch  8/290
Loss: 1.4279738664627075
Working on batch  9/290
Loss: 2.5996053218841553
Working on batch  10/290
Loss: 2.011125326156616
Working on batch  11/290
Loss: 2.6865453720092773
Working on batch  12/290
Loss: 1.8920385837554932
Working on batch  13/290
Loss: 1.9380030632019043
Working on batch  14/290
Loss: 2.3911614418029785
Working on batch  15/290
Loss: 1.803059458732605
Working on batch  16/290
Loss: 1.412466049194336
Working on batch  17/290
Loss: 1.6524972915649414
Working on batch  18/290
Loss: 1.6621967554092407
Working on batch  19/290
Loss: 1.81128990650177
Working on batch  20/290
Loss: 1.62137091159

In [ ]:
def plot_roc_curve(dataloader):
    model.eval()
    test_probs = []
    test_targets = []
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            test_probs.extend(probabilities[:, 1].cpu().numpy())
            test_targets.extend(labels.cpu().numpy())

    fpr, tpr, _ = roc_curve(test_targets, test_probs)
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
timm.create_model()